In [128]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import tkinter as tk
from tkinter import messagebox, filedialog
from PIL import Image, ImageTk


class MLPGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("GROUP 3")

        # Background Image
        bg_image = Image.open("background.png")
        bg_image = bg_image.resize((950, 600), Image.Resampling.LANCZOS)
        self.bg_photo = ImageTk.PhotoImage(bg_image)
        self.canvas = tk.Canvas(self.master, width=950, height=600)
        self.canvas.pack(fill="both", expand=True)
        self.canvas.create_image(0, 0, image=self.bg_photo, anchor="nw")

        # Load Dataset and Train Model Buttons
        self.file_button = tk.Button(self.master, text="Load Dataset", command=self.load_file)
        self.file_button.place(relx=0.35, rely=0.05, anchor="n")

        self.file_status_label = tk.Label(self.master, text="✘ Dataset Not Loaded", fg="red", bg="white")
        self.file_status_label.place(relx=0.5, rely=0.05, anchor="n")

        self.train_button = tk.Button(self.master, text="Train Model", command=self.train_model, state=tk.DISABLED)
        self.train_button.place(relx=0.65, rely=0.05, anchor="n")

        # R² Score Label
        self.r2_label = tk.Label(self.master, text="R² Score: -------", font=("Arial", 12), bg="white")
        self.r2_label.place(relx=0.5, rely=0.2, anchor="n")

        # Enter Data for Prediction
        self.input_label = tk.Label(self.master, text="Enter Data for Prediction:", font=("Arial", 12), bg="white")
        self.input_label.place(relx=0.5, rely=0.3, anchor="n")

        self.inputs_frame = tk.Frame(self.master, bg="white")
        self.inputs_frame.place(relx=0.5, rely=0.4, anchor="n")

        self.inputs = {}
        self.feature_names = []

        # Prediction Result Label
        self.results_label = tk.Label(self.master, text="Prediction for Total PTS: -------", font=("Arial", 12), bg="white")
        self.results_label.place(relx=0.5, rely=0.8, anchor="n")

        self.predict_button = tk.Button(self.master, text="Predict", command=self.predict, state=tk.DISABLED)
        self.predict_button.place(relx=0.5, rely=0.7, anchor="n")

        # Variables to hold model and data
        self.mlp_model = None
        self.scaler = None
        self.file_loaded = False

    def load_file(self):
        try:
            file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
            if file_path:
                self.data = pd.read_csv(file_path)
                self.file_status_label.config(text="✔ Dataset Loaded", fg="green")
                self.file_loaded = True
                self.train_button.config(state=tk.NORMAL)
            else:
                self.file_status_label.config(text="✘ Dataset Not Loaded", fg="red")
                self.file_loaded = False
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while loading the file: {e}")

    def train_model(self):
        try:
            if not self.file_loaded:
                messagebox.showerror("Error", "Please load a dataset first.")
                return

            numeric_data = self.data.select_dtypes(include=['float64', 'int64'])
            X = numeric_data.drop(columns=['pts'])
            y = numeric_data['pts']
            X = X.dropna()
            y = y[X.index]

            # Feature selection using RandomForest
            model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
            model_rf.fit(X, y)
            feature_importances = pd.Series(model_rf.feature_importances_, index=X.columns).sort_values(ascending=False)

            # Select optimal features
            threshold = 0.97
            cumulative_importances = feature_importances.cumsum()
            self.feature_names = feature_importances[cumulative_importances <= threshold].index.tolist()

            # Prepare data with selected features
            X_selected = X[self.feature_names]
            self.scaler = MinMaxScaler()
            X_normalized = self.scaler.fit_transform(X_selected)

            X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

            # Train MLP model
            self.mlp_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
            self.mlp_model.fit(X_train, y_train)

            # Test model
            y_pred = self.mlp_model.predict(X_test)
            r2 = r2_score(y_test, y_pred)

            # Update R² score label
            self.r2_label.config(text=f"R² Score: {r2:.4f}")

            # Create input fields for features
            for widget in self.inputs_frame.winfo_children():
                widget.destroy()

            self.inputs = {}
            for feature in self.feature_names:
                label = tk.Label(self.inputs_frame, text=f"{feature}: ", bg="white")
                label.grid(row=0, column=self.feature_names.index(feature), padx=5, pady=5)

                entry = tk.Entry(self.inputs_frame)
                entry.grid(row=1, column=self.feature_names.index(feature), padx=5, pady=5)
                self.inputs[feature] = entry

            self.predict_button.config(state=tk.NORMAL)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def predict(self):
        try:
            # Collect input data
            input_data = []
            for feature in self.feature_names:
                value = float(self.inputs[feature].get())
                input_data.append(value)

            # Normalize input data
            input_data_normalized = self.scaler.transform([input_data])

            # Predict
            prediction = self.mlp_model.predict(input_data_normalized)[0]

            # Update results label
            self.results_label.config(text=f"Prediction for Total PTS: {prediction:.4f}")

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values for all features.")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during prediction: {e}")


def center_window(window, width=1024, height=768):
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)
    window.geometry(f"{width}x{height}+{x}+{y}")


if __name__ == "__main__":
    root = tk.Tk()
    center_window(root, 950, 600)
    app = MLPGUI(root)
    root.mainloop()


C:\Users\yasin\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
